# Python Optimization

In this section we will discuss python optimization.  The key fact about python optimization is several-fold.
1. Is your code correct?
2. Do you need to optimize?
3. Do you really need to optimize?
4. Optimize is not parallelization -- usually do this last.
5. Optimization involves tradeoffs.  Be careful what you wish for.

There are a few steps to optimization:
1. profile
2. profile again.
3. check the hotspots.
4. payoff in optimization: modify your use case, use better algorithms, use builtin functions, use numba, pre-compiled code



## Profiling

The first and most important aspect of optimization is to figure out what is the slow part.  For this you need to profile your code.  Fortunately python offers some excellent profilers and jupyter has step this part up even further.  For this, we will use the line decorated %prun.  

In [2]:
import numpy as np 
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
N_massive = 20
N_bodies = N_massive
M = np.ones(N_bodies)

def Nbody_derivatives(pos,vel) :
    dpdt = vel
    dvdt = np.zeros(vel.shape)
    for i in range(N_bodies) :
        for j in range(N_bodies) :
            if i == j : 
                continue
            r = np.linalg.norm( pos[j]-pos[i])
            mass = M[j]
            rhat =(pos[j] - pos[i])/r
            dvdt[i] += mass/(r*r)*rhat
        
    return dpdt, dvdt

def initial_conditions() : 
    pos = np.random.random([N_bodies,3])
    vel = np.random.random([N_bodies,3])

    return pos, vel

def run_Nbody_rk2(tend,tframe,dt,derivatives=Nbody_derivatives) :
    p,v = initial_conditions()
    t = 0
    tnext = tframe
    positions = []
    while t<tend :
        while t < tnext :
            delta_t = min(tnext-t,dt)
            dpdt, dvdt = derivatives(p,v) 
            phalf, vhalf = p+dpdt*0.5*delta_t, v+dvdt*0.5*delta_t
            dpdt, dvdt = derivatives(phalf, vhalf)
            p, v = p + dpdt*delta_t, v + dvdt*delta_t
            t += delta_t
        positions.append(p.copy())
        tnext += tframe
    return positions

tframe = 0.01
dt = 0.001
frames = 100
%prun positions = run_Nbody_rk2(frames*tframe, tframe, dt)
from matplotlib.animation import FuncAnimation

def animate(i, positions):
    ax.clear()
    # Get the point from the points list at index i
    pos = positions[i]
    ax.scatter(pos[:,0], pos[:,1], color='green', marker='o')
    # Set the x and y axis to display a fixed range
    ax.set_xlim([-100,100])
    ax.set_ylim([-100,100])
fig, ax = plt.subplots()
ani = FuncAnimation(fig, lambda i : animate(i, positions), frames=len(positions), interval=50, repeat=False)
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()


As you can see the %prun reveals what is slow.  Nearly the entire code is spent in Nbody_derivatives, which computes the accelerations.  So it goes to show what profiling can do for you.  Pretty much nothing else is spent anywhere else. 

## Optimal python

The first thing we want to do is rewrite the code so that it is more correct or more pythonic -- this means that we want to write it so that is more numpy like.  Lets look at the following.  Starting with the above code, I challenge you to write it so that it is significantly cleaner.

In [ ]:
def Nbody_derivatives2(pos,vel) :
    dpdt = vel
    dvdt = np.zeros(vel.shape)
    # write the above in optimal python using numpy.  How fast can you make it.
    return dpdt, dvdt

%prun positions = run_Nbody_rk2(frames*tframe, tframe, dt, derivatives=Nbody_derivatives2)


fig, ax = plt.subplots()
ani = FuncAnimation(fig, lambda i : animate(i, positions), frames=len(positions), interval=50, repeat=False)
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()


This was a significant speedup -- about a factor of 20.   This is really excellent.

## Hot spot optimization with numba

Recently python compilation has started to become a thing.  One noteworthy example is numba, which is a jit compiler that works well with numpy. Lets try this one, but using the code originally.  See if you code is faster.

In [14]:
from numba import jit,njit
@jit
def Nbody_derivatives3(pos,vel) :
    dpdt = vel
    dvdt = np.zeros(vel.shape)
    for i in range(N_bodies) :
        for j in range(N_bodies) :
            if i == j : 
                continue
            r = np.linalg.norm( pos[j]-pos[i])
            mass = M[j]
            rhat =(pos[j] - pos[i])/r
            dvdt[i] += mass/(r*r)*rhat
        
    return dpdt, dvdt

%prun positions = run_Nbody_rk2(frames*tframe, tframe, dt, derivatives=Nbody_derivatives3)
fig, ax = plt.subplots()
ani = FuncAnimation(fig, lambda i : animate(i, positions), frames=frames, interval=25, repeat=False)
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()


This is faster than idiomic python, but it can be marginally so. Still much faster than the original version.  Not bad for a simple @jit

## Hot spot optimization with cython


In [9]:
%load_ext cython

In [12]:
%%cython -a
import numpy as np
from cpython cimport array
import array
import math 
def cython_Nbody_derivatives(pos,vel,M) :
    cdef int N_bodies = M.size

    dpdt = vel
    dvdt_arr = np.zeros([N_bodies,3])
    cdef double [:, :] dvdt = dvdt_arr
    cdef double r = 0., mass = 0.
    cdef double [:,:] p = pos
    cdef double rh = 0.
    for i in range(N_bodies) :
        for j in range(N_bodies) :
            if i == j : 
                continue
            r = 0.
            for k in range(3) : 
                r += ( p[j,k]-p[i,k])*( p[j,k]-p[i,k])
            r = math.sqrt(r)
            mass = M[j]
            for k in range(3) :
                rh =(p[j,k] - p[i,k])/r
                dvdt[i,k] += mass/(r*r)*rh
        
    return dpdt, dvdt_arr


In [13]:
def cython_derivs(pos,vel) :
    return cython_Nbody_derivatives(pos,vel,M)
%prun positions = run_Nbody_rk2(frames*tframe, tframe, dt, derivatives=cython_derivs)
fig, ax = plt.subplots()
ani = FuncAnimation(fig, lambda i : animate(i, positions), frames=frames, interval=25, repeat=False)
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()


## Hot spot optimization with f2py

The final example is using fortran to optimize the slowest bits.  Why fortran?  Because fortran 90 plays extremely well with python.

First we generate the .f90 file.

In [5]:
%%file nbody_derivatives.f90
SUBROUTINE derivs(pos,vel,mass,dpdt,dvdt,n)
    implicit none
    integer, intent(IN) :: n
    double precision, intent(IN), dimension(n,3):: pos, vel
    double precision, intent(IN), dimension(n) :: mass
    double precision, intent(OUT), dimension(n,3):: dpdt, dvdt
!f2py intent(in) n
!f2py intent(in) pos, vel, mass
!f2py intent(out) dpdt, dvdt
!f2py depend(n) mass
    integer :: i, j
    double precision, dimension(3) :: rhat,r
    double precision :: r2
    
    dpdt(:,:) = vel(:,:)
    dvdt(:,:) = 0.
    do i = 1,n
        do j = 1,n
            if( i .eq. j) then
                cycle
            endif
            r(:) = pos(j,:) - pos(i,:)
            r2 = sum(r*r)
            rhat = r/sqrt(r2)
            dvdt(i,:) = dvdt(i,:)+ mass(j)/(r2)*rhat(:)
        enddo
    enddo
    
    return
end subroutine derivs

Writing nbody_derivatives.f90


In [6]:
!f2py -m nbody_derivatives -c nbody_derivatives.f90

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "nbody_derivatives" sources
f2py options: []
f2py:> /tmp/tmp232jcx96/src.linux-x86_64-3.7/nbody_derivativesmodule.c
creating /tmp/tmp232jcx96/src.linux-x86_64-3.7
Reading fortran codes...
	Reading file 'nbody_derivatives.f90' (format:free)
Post-processing...
	Block: nbody_derivatives
			Block: derivs
Post-processing (stage 2)...
Building modules...
	Building module "nbody_derivatives"...
		Constructing wrapper function "derivs"...
		  dpdt,dvdt = derivs(pos,vel,mass,[n])
	Wrote C/API module "nbody_derivatives" to file "/tmp/tmp232jcx96/src.linux-x86_64-3.7/nbody_derivativesmodule.c"
  adding '/tmp/tmp232jcx96/src.linux-x86_64-3.7/fortranobject.c' to sources.
  adding '/tmp/tmp232jcx96/src.linux-x86_64-3.7' to include_dirs.
c

In [15]:
import nbody_derivatives as nbd
import importlib
importlib.reload(nbd)

def fortran_derivs(pos,vel) :
    return nbd.derivs(pos,vel,M)
%prun positions = run_Nbody_rk2(frames*tframe, tframe, dt, derivatives=fortran_derivs)
#print(positions)
fig, ax = plt.subplots()
ani = FuncAnimation(fig, lambda i : animate(i, positions), frames=frames, interval=50, repeat=False)
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()


This is extremely fast like 600x faster than the original code.  Compile code in a highly optimized language is extremely powerful.  

# Running in Parallel



In [ ]:
import multiprocessing 
count = 4
pool = multiprocessing.Pool(processes=count)

def onebody_derivatives(i, pos, vel) :
    dvdt = np.zeros(3)
    for j in range(N_bodies) :
        if i == j : 
            continue
        r = np.linalg.norm( pos[j]-pos[i])
        mass = M[j]
        rhat =(pos[j] - pos[i])/r
        dvdt += mass/(r*r)*rhat
        
    return dvdt

def parallel_Nbody_derivatives(pos,vel) :
    dpdt = vel
    res = pool.starmap(onebody_derivatives, zip(range(N_bodies), [pos] * N_bodies, [vel]*N_bodies))
    dvdt = np.array(res)
    return dpdt, dvdt

%prun positions = run_Nbody_rk2(frames*tframe, tframe, dt, derivatives=parallel_Nbody_derivatives)
fig, ax = plt.subplots()
ani = FuncAnimation(fig, lambda i : animate(i, positions), frames=frames, interval=50, repeat=False)
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()


This is the problem with parallelization.  It doesn't usually give you the best performance.  Even this one where the operation is relatively parallel, the gains are not so great as being more careful about how you approach things.  